In [53]:
import duckdb
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
from transformers import AutoTokenizer, DistilBertModel, BigBirdModel, BigBirdTokenizerFast , DistilBertTokenizerFast
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [ ]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_vector (
    ID INT PRIMARY KEY,
    content TEXT,
  );
""")

In [ ]:
concedendo_label = 1
negando_label = 0

In [3]:
data = con.execute("SELECT * FROM decisions_description").fetchnumpy()
data

{'ID': array([121184492, 122027196, 123312824, ..., 143346074, 143349037,
        143349119], dtype=int32),
 'content': array(['DECISION > It is treated of habeas corpus impetrado for the benefit of Valter de Oliveira Albano - fulfilling penalty in the Center for Resocialization of Lins/SP, currently in closed regime, with the end of the sentence provided for 19/3/2027 -, pointing itself as coactor body maintaining the Court of Justice of São Paulo, which ordered the HC no. 21346-19-77.20 The judgment has this issue (f. 82):<br>HABEAS CORPUS - CRIMINAL EXECUTION - Impetration aiming at granting home imprisonment by integrating risk group, carrier of hypertension - Impossibility - Absence of cabal proof that the prison unit that houses it does not have adequate structure to keep it, nor to offer it,47 if necessary, the appropriate treatment Recommendation No 62 of the CNJ which has no binding character - Implementation of measures To contain the spread of the disease in prison units - S

In [16]:
data["ID"].size

3092

In [51]:
# Vamos usar numpy.vectorize para aplicar len (comprimento de string) a cada elemento do array
len_vectorized = np.vectorize(len)

# Aplicar a função vetorizada ao array de strings para obter um array de comprimentos
array_len = len_vectorized(data["content"])

# Encontrar o índice do maior comprimento
index_of_max = np.argmax(array_len)

# Usar o índice para encontrar a string mais longa
longest_str = data["content"][index_of_max]

len(longest_str)

186592

In [42]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=512, chunk_overlap=10
)

In [52]:
texts = text_splitter.split_text(longest_str)
print(len(texts))

inputs = tokenizer(texts[0], return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

print(last_hidden_states.shape)

104
torch.Size([1, 366, 768])


In [55]:
tokenizer = BigBirdTokenizerFast.from_pretrained("google/bigbird-roberta-base")
model = BigBirdModel.from_pretrained("google/bigbird-roberta-base",attention_type="block_sparse")

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=4096, chunk_overlap=10
)

In [58]:
texts = text_splitter.split_text(longest_str)
print(len(texts))

inputs = tokenizer(texts[0], return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

print(last_hidden_states.shape)

13
torch.Size([1, 3806, 768])


In [ ]:
for index, element in enumerate(data["ID"]):
  print([index])
  out_str = summarizer.summarize_string(data["content"][index])
  con.execute("""
    UPDATE decisions_description
    SET summarize = ?
    WHERE ID = ?;
    """,(out_str,index))

In [59]:
con.close()